In [1]:
import torch
from torch.nn import BCELoss
import torch.nn as nn
import torch.optim.lr_scheduler as lrsch


from importlib import reload

import cnn
import helpers
reload(cnn)
reload(helpers)
from helpers import ClassifierMetrics, display_tensor_as_rgb
from cnn import LateFusionCNN, train_multicrop
from data import load_data
import warnings
warnings.filterwarnings('ignore')

from constants import *
from datasets import enterface
from loss import multicrop_loss

In [2]:
lfc = LateFusionCNN(enterface)
lfc = nn.DataParallel(lfc).cuda()

mbt_teacher = LateFusionCNN(enterface)
mbt_teacher = nn.DataParallel(mbt_teacher).cuda()

mbt_student = LateFusionCNN(enterface)
mbt_student = nn.DataParallel(mbt_student).cuda()

train_dl, val_dl, test_dl, ss  = load_data(enterface, 
                                       batch_sz=BATCH_SZ,
                                       train_val_test_split=SPLIT,
                                       force_audio_aspect=True)

print(f"split_seed: {ss}")
optimizer = torch.optim.AdamW(lfc.parameters(), betas=(0.9, 0.999), lr=LR, weight_decay=1.0 / BATCH_SZ)
scheduler = lrsch.SequentialLR(
    optimizer=optimizer,
    schedulers=[
    lrsch.ConstantLR(optimizer=optimizer, factor=1, total_iters=WARMUP_EPOCHS),
    lrsch.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=T_0)
], milestones=MILESTONES)

loss_func = BCELoss()

split_seed: 5651295578886405114


In [3]:
reload(cnn)

updated_centre = CENTRE_CONSTANT
for epoch in range(EPOCHS):
    train_loss, updated_centre = train_multicrop(mbt_teacher, mbt_student, train_dl, optimizer, updated_centre, loss_fn=multicrop_loss)
    scheduler.step()
    print(f"Epoch {epoch + 1}: train_loss {train_loss:.5f}\n")

  0%|          | 0/161 [00:00<?, ?it/s]

KeyboardInterrupt: 